In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path = '/content/drive/MyDrive/deteksi-gambar/data/training'

In [ ]:
def get_model(IMG_SIZE):
    model = tf.keras.models.Sequential([
        # Model layers
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    model.summary()

In [ ]:
#store keselurhan predictor
main_pred = []
error = []

In [ ]:
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 100
N_SPLIT = 5

In [ ]:
#menetapkan nilai per-foldnya
acc_per_fold = []
loss_per_fold = []

In [ ]:
#inisialisasi generator data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#inisialisasi k-fold
kfold = StratifiedKFold(n_splits=N_SPLIT,
                        shuffle = True,
                        random_stats = 42)

In [ ]:
#variable menghitung setiap pembagiaannya
j = 0

In [ ]:
#kfold traing dan test untuk setiap pembagiannya
for train_idx, val_idx in list(kfold.split(train_path, None)):
    j += 1

    training_set = train_datagen.flow_from_directory(train_path,
                                                     target_size=(IMG_SIZE, IMG_SIZE),
                                                     batch_size=BATCH_SIZE,
                                                     class_mode="binary",
                                                     subset='training',
                                                     shuffle=True)
    validation_set = validation_datagen.flow_from_directory(train_path,
                                                            target_size=(IMG_SIZE, IMG_SIZE),
                                                            batch_size=BATCH_SIZE,
                                                            class_mode="binary",
                                                            subset='validation',
                                                            shuffle=False)

    model = get_model(IMG_SIZE)

    history = model.fit(training_set,
                        validation_data=validation_set,
                        epochs=EPOCHS,
                        steps_per_epoch=training_set.samples // BATCH_SIZE,
                        validation_steps=validation_set.samples // BATCH_SIZE)

    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.title(f'Model Accuracy and Loss Fold {j}')
    plt.ylabel('val')
    plt.xlabel('No. Epoch')
    plt.legend(loc="upper left")
    plt.show()

    scores = model.evaluate(validation_set, verbose=0)

    pred = model.predict(validation_set)
    y_pred = np.argmax(pred, axis=1)

    cf_matrix = confusion_matrix(validation_set.classes, y_pred)
    f, ax = plt.subplot(figsize=(9, 6))
    sns.heatmap(cf_matrix, annot=True, fmt="d", linewidths=.5, ax=ax)
    plt.xlabel('y_pred')
    plt.ylabel('y_true')
    ax.xaxis.set_ticklabels(['organic', 'inorganic'])
    ax.yaxis.set_ticklabels(['organic', 'inorganic'])
    plt.show()

    model.save(f'/content/drive/MyDrive/deteksi-gambar/model/model{j}.h5')
    print(f'Score for fold {j}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1]*100)
    loss_per_fold.append(scores[0])